In [287]:
"""

a faire : prendre une nouvelle BDD et garder un maximum d'options mais diverses, essayer de calibrer sans le vega  
"""


import numpy as np
import pandas as pd
from scipy.optimize import least_squares
import Heston_pricer as hp  

In [288]:
#étape1 : récupérer les données de marchés (que des calls)
df = pd.read_excel(r"C:\Users\malu\Documents\Perso\Memoire\Vdef\BDD.xlsx") # Change this path!!!!!!
df = df[df["Type"].str.lower() == "call"] # On ne garde que les calls 
df

,Sous-jacent,Type,Prix d'exercice,Maturité,Bid,Ask,Spot
0,Amazon,call,240,2025-07-18,0.070,0.08,219
1,Amazon,call,230,2025-07-18,0.270,0.28,219
2,Amazon,call,220,2025-07-18,0.850,0.88,219
3,Amazon,call,250,2025-07-18,0.001,0.05,219
4,Amazon,call,255,2025-08-15,0.210,0.22,219
...,...,...,...,...,...,...,...
457,S&P 500,Call,6100,2025-07-18,1.460,1.45,6254
458,S&P 500,Call,6000,2025-07-18,2.270,2.26,6254
459,S&P 500,Call,5900,2025-07-18,3.110,3.10,6254
460,S&P 500,Call,5800,2025-07-18,3.950,3.95,6254


In [289]:
#Ajout d'une colonne mid qui nous sert de prix
df ['Mid']=(df['Bid']+df['Ask'])/2
df.head()

,Sous-jacent,Type,Prix d'exercice,Maturité,Bid,Ask,Spot,Mid
0,Amazon,call,240,2025-07-18,0.070,0.08,219,0.0750
1,Amazon,call,230,2025-07-18,0.270,0.28,219,0.2750
2,Amazon,call,220,2025-07-18,0.850,0.88,219,0.8650
3,Amazon,call,250,2025-07-18,0.001,0.05,219,0.0255
4,Amazon,call,255,2025-08-15,0.210,0.22,219,0.2150


In [290]:
#étape2 : calculer la vol implicite pour chacune de ces options de marchés 

import Implied_volatility as iv

reference_date = pd.Timestamp('2025-07-01') # date à laquelle les données ont été extraites (sur le site de la société générale)

df['T'] = (df['Maturité'] - reference_date).dt.days / 365.0

r=0.02 # risk free

# Calcul de la volatilité implicite des options sur le marché
df['VI_market'] = df.apply(
    lambda row: iv.call_implied_volatility_mc(
        row['Mid'], 
        row['Spot'], 
        row['Prix d\'exercice'], 
        row['T'], 
        r
    ), 
    axis=1
)
#df.dropna(inplace=True) # Supprime les lignes avec NaN
df

,Sous-jacent,Type,Prix d'exercice,Maturité,Bid,Ask,Spot,Mid,T,VI_market
0,Amazon,call,240,2025-07-18,0.070,0.08,219,0.0750,0.046575,0.204805
1,Amazon,call,230,2025-07-18,0.270,0.28,219,0.2750,0.046575,0.158086
2,Amazon,call,220,2025-07-18,0.850,0.88,219,0.8650,0.046575,0.064673
3,Amazon,call,250,2025-07-18,0.001,0.05,219,0.0255,0.046575,0.244612
4,Amazon,call,255,2025-08-15,0.210,0.22,219,0.2150,0.123288,0.226159
...,...,...,...,...,...,...,...,...,...,...
457,S&P 500,Call,6100,2025-07-18,1.460,1.45,6254,1.4550,0.046575,NaN
458,S&P 500,Call,6000,2025-07-18,2.270,2.26,6254,2.2650,0.046575,NaN
459,S&P 500,Call,5900,2025-07-18,3.110,3.10,6254,3.1050,0.046575,NaN
460,S&P 500,Call,5800,2025-07-18,3.950,3.95,6254,3.9500,0.046575,NaN


In [291]:
# étape 3 : simuler le prix des options de marchés avec le modèle de Heston 

r=0.02 # risk free

# paramètres initiaux, non calibrés
sigma = 0.5
kappa = 1
theta= 0.05
volvol= 0.025
rho =-0.5


# Calcl du prix avec le modèle de Heston avec les paramètres non calibré
df["Heston_price"] = df.apply(
    lambda row: hp.call_priceHestonMid(
        row['Spot'],
        row["Prix d'exercice"],
        r,
        row["T"],
        sigma,
        kappa,
        theta,
        volvol,
        rho
    ),
    axis=1
)

df

,Sous-jacent,Type,Prix d'exercice,Maturité,Bid,Ask,Spot,Mid,T,VI_market,Heston_price
0,Amazon,call,240,2025-07-18,0.070,0.08,219,0.0750,0.046575,0.204805,-0.063176
1,Amazon,call,230,2025-07-18,0.270,0.28,219,0.2750,0.046575,0.158086,0.169928
2,Amazon,call,220,2025-07-18,0.850,0.88,219,0.8650,0.046575,0.064673,2.546705
3,Amazon,call,250,2025-07-18,0.001,0.05,219,0.0255,0.046575,0.244612,-0.129630
4,Amazon,call,255,2025-08-15,0.210,0.22,219,0.2150,0.123288,0.226159,-0.146994
...,...,...,...,...,...,...,...,...,...,...,...
457,S&P 500,Call,6100,2025-07-18,1.460,1.45,6254,1.4550,0.046575,NaN,190.707881
458,S&P 500,Call,6000,2025-07-18,2.270,2.26,6254,2.2650,0.046575,NaN,273.799255
459,S&P 500,Call,5900,2025-07-18,3.110,3.10,6254,3.1050,0.046575,NaN,365.372034
460,S&P 500,Call,5800,2025-07-18,3.950,3.95,6254,3.9500,0.046575,NaN,461.495106


In [292]:
# Garder uniquement les options sur le S&P 500 avec des prix valides
df = df[
    (df['Heston_price'] > 0) &
    (df['Mid'] > 0) &
    (df['Sous-jacent '].str.strip() == 'S&P 500')
]

# Calcul de la moneyness : Strike / Spot (pour call)
df['moneyness'] = df["Prix d'exercice"] / df["Spot"]

# Catégorisation selon la moneyness
df['Classe'] = pd.cut(
    df['moneyness'],
    bins=[0, 0.85, 0.95, 1.05, 1.15, float('inf')],
    labels=['Deep ITM', 'ITM', 'ATM', 'OTM', 'Deep OTM']
)

# Suppression des classes manquantes pour éviter les erreurs
df = df.dropna(subset=['Classe'])

# Échantillonnage équilibré (par exemple 20% de chaque classe)
df_sample = (
    df.groupby('Classe', group_keys=False)
    .apply(lambda x: x.sample(frac=0.2, random_state=42) if len(x) >= 5 else x)
    .reset_index(drop=True)
)


C:\Users\malu\AppData\Local\Temp\ipykernel_11220\2188228279.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['moneyness'] = df["Prix d'exercice"] / df["Spot"]
C:\Users\malu\AppData\Local\Temp\ipykernel_11220\2188228279.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Classe'] = pd.cut(
C:\Users\malu\AppData\Local\Temp\ipykernel_11220\2188228279.py:23: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to ret

In [293]:
# étape 4 : Obtenir la vol implicte des options calculé avec le modèle de Heston
df['VI_heston'] = df.apply(
    lambda row: iv.call_implied_volatility_mc(
        row['Heston_price'], 
        row['Spot'], 
        row['Prix d\'exercice'], 
        row['T'], 
        r
    ), 
    axis=1
)
#df.dropna(inplace=True)
df

,Sous-jacent,Type,Prix d'exercice,Maturité,Bid,Ask,Spot,Mid,T,VI_market,Heston_price,moneyness,Classe,VI_heston
413,S&P 500,Call,6800,2026-03-20,1.28,1.28,6254,1.280,0.717808,0.038539,141.123245,1.087304,OTM,0.144561
414,S&P 500,Call,6600,2026-03-20,2.00,2.00,6254,2.000,0.717808,0.025901,210.417347,1.055325,OTM,0.148866
415,S&P 500,Call,6400,2026-03-20,2.92,2.93,6254,2.925,0.717808,0.008920,302.917231,1.023345,ATM,0.155349
416,S&P 500,Call,6000,2026-03-20,5.24,5.24,6254,5.240,0.717808,NaN,549.927060,0.959386,ATM,0.172634
417,S&P 500,Call,5600,2026-03-20,7.94,7.93,6254,7.935,0.717808,NaN,855.129501,0.895427,ITM,0.191914
418,S&P 500,Call,6800,2025-12-19,0.64,0.65,6254,0.645,0.468493,0.046180,78.222301,1.087304,OTM,0.140192
419,S&P 500,Call,6700,2025-12-19,0.88,0.89,6254,0.885,0.468493,0.039734,102.890092,1.071314,OTM,0.141612
420,S&P 500,Call,6600,2025-12-19,1.19,1.19,6254,1.190,0.468493,0.032417,134.084288,1.055325,OTM,0.143846
421,S&P 500,Call,6500,2025-12-19,1.57,1.57,6254,1.570,0.468493,0.023995,172.491221,1.039335,ATM,0.146930
422,S&P 500,Call,6400,2025-12-19,2.01,2.02,6254,2.015,0.468493,0.013913,218.298366,1.023345,ATM,0.150792


In [294]:
# étape 5 : pour chacune des options calculer le véga 
from scipy.stats import norm
import numpy as np

def black_scholes_vega(S, K, T, r, sigma):
    if T <= 0 or sigma <= 0:
        return 0.0
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    return S * norm.pdf(d1) * np.sqrt(T)

df['Vega'] = df.apply(lambda row: black_scholes_vega(row['Spot'], row["Prix d'exercice"], row["T"], r, row['VI_market']), axis=1)
df

,Sous-jacent,Type,Prix d'exercice,Maturité,Bid,Ask,Spot,Mid,T,VI_market,Heston_price,moneyness,Classe,VI_heston,Vega
413,S&P 500,Call,6800,2026-03-20,1.28,1.28,6254,1.280,0.717808,0.038539,141.123245,1.087304,OTM,0.144561,229.414550
414,S&P 500,Call,6600,2026-03-20,2.00,2.00,6254,2.000,0.717808,0.025901,210.417347,1.055325,OTM,0.148866,426.921275
415,S&P 500,Call,6400,2026-03-20,2.92,2.93,6254,2.925,0.717808,0.008920,302.917231,1.023345,ATM,0.155349,1091.021146
416,S&P 500,Call,6000,2026-03-20,5.24,5.24,6254,5.240,0.717808,NaN,549.927060,0.959386,ATM,0.172634,NaN
417,S&P 500,Call,5600,2026-03-20,7.94,7.93,6254,7.935,0.717808,NaN,855.129501,0.895427,ITM,0.191914,NaN
418,S&P 500,Call,6800,2025-12-19,0.64,0.65,6254,0.645,0.468493,0.046180,78.222301,1.087304,OTM,0.140192,111.594030
419,S&P 500,Call,6700,2025-12-19,0.88,0.89,6254,0.885,0.468493,0.039734,102.890092,1.071314,OTM,0.141612,160.467220
420,S&P 500,Call,6600,2025-12-19,1.19,1.19,6254,1.190,0.468493,0.032417,134.084288,1.055325,OTM,0.143846,234.147015
421,S&P 500,Call,6500,2025-12-19,1.57,1.57,6254,1.570,0.468493,0.023995,172.491221,1.039335,ATM,0.146930,356.320564
422,S&P 500,Call,6400,2025-12-19,2.01,2.02,6254,2.015,0.468493,0.013913,218.298366,1.023345,ATM,0.150792,610.254123


In [295]:
# étape  6 : établir la fonction à minimiser (différence entre les vols implicite pondéré par le vega)

def residuals_vega_weighted(params, df, r=0.02):
    kappa, theta, sigma, rho, volvol = params
    res = []

    for i, row in df.iterrows():
        try:
            S = row['Spot']
            K = row["Prix d'exercice"]
            T = row['T']
            market_price = row['Mid']

            # --- Calcul du prix modèle (Heston) ---
            model_price = hp.call_priceHestonMid(S, K, r, T, sigma, kappa, theta, volvol, rho)

            # --- Calcul du vega Black-Scholes ---
            vega = black_scholes_vega(S, K, T, r, row['VI_market'])  

            # Si vega ou prix invalide, on pénalise fortement
            if not np.isfinite(model_price) or not np.isfinite(vega) or vega < 1e-8:
                res.append(1e6)
            else:
                res.append(np.sqrt(vega) * (model_price - market_price)) # On utilise la racine carrée du vega car la fonction least square élève au carré la fonction résidu

        except Exception as e:
            print(f"Erreur ligne {i}: {e}")
            res.append(1e6)

    return np.array(res)


In [296]:
#étape 6 bis : on fait aussi une fonction pour essayer de calibrer sur les prix et pas sur la volatilité 

def residuals(params, df, r=0.02):
    kappa, theta, sigma, rho, volvol = params
    res = []

    for i, row in df.iterrows():
        try:
            S = row['Spot']
            K = row["Prix d'exercice"]
            T = row['T']
            market_price = row['Mid']

            # --- Calcul du prix modèle (Heston) ---
            model_price = hp.call_priceHestonMid(S, K, r, T, sigma, kappa, theta, volvol, rho)
        
            res.append((model_price/100 - market_price)) 
        except Exception as e:
            print(f"Erreur ligne {i}: {e}")
            res.append(1e6)
    return np.array(res)

In [297]:
from scipy.optimize import least_squares

#Paramètres initiaux 
init_params = [1.0, 0.04, 0.5, -0.5, 0.04]  # [kappa, theta, sigma, rho, volvol]

# --- Lancement de la calibration ---
print("Lancement de la calibration ")

result = least_squares(
    residuals,
    init_params,
    args=(df,),
    method='lm',          # méthode Levenberg-Marquardt
    verbose=2
)

# --- Résultats ---
kappa, theta, sigma, rho, volvol = result.x
rmse = np.sqrt(np.mean(result.fun**2))

print("\n Calibration terminée.")
print(f"Paramètres calibrés :\n kappa={kappa:.4f}, theta={theta:.4f}, "
      f"sigma={sigma:.4f}, rho={rho:.4f}, volvol={volvol:.4f}")
print(f"RMSE : {rmse:.4f}")


Lancement de la calibration 
`xtol` termination condition is satisfied.
Function evaluations 98, initial cost 1.3140e+01, final cost 3.2594e+00, first-order optimality 3.16e-01.

 Calibration terminée.
Paramètres calibrés :
 kappa=3.1927, theta=0.0321, sigma=0.0002, rho=-0.9616, volvol=0.0070
RMSE : 0.3647
